* initial construction to collision resolution

In [ ]:
import unittest



*   variant reads (sort by coverage?)



In [2]:
def generate_sequence(n):
  import random

  bases = ['a','g','c','t']
  seq = ''
  for i in range(n):
    seq += bases[random.randint(0,3)]
  return seq

def chop_sequence(sequence,min_len=3,max_len=15,min_overlap=3):
  import random

  start = 0
  end = max(random.randint(start,max_len),min_len)
  chop = [sequence[start:end]]
  while end < len(sequence):
    start = random.randint(start + 1,end - min_overlap)
    end = max(random.randint(start,start + max_len),start + min_len)
    chop += [sequence[start:end]]
  return chop

In [6]:
ref = "Fresh out of college, Barry the Bee (Jerry Seinfeld) finds the prospect of working with honey uninspiring. He flies outside the hive for the first time and talks to a human (Renée Zellweger), breaking a cardinal rule of his species. Barry learns that humans have been stealing and eating honey for centuries, and he realizes that his true calling is to obtain justice for his kind by suing humanity for theft." #generate_sequence(200)
reads = chop_sequence(ref,9,18)
import random
# random.shuffle(reads)
print(ref)
reads

Fresh out of college, Barry the Bee (Jerry Seinfeld) finds the prospect of working with honey uninspiring. He flies outside the hive for the first time and talks to a human (Renée Zellweger), breaking a cardinal rule of his species. Barry learns that humans have been stealing and eating honey for centuries, and he realizes that his true calling is to obtain justice for his kind by suing humanity for theft.


['Fresh out',
 'resh out ',
 'sh out of',
 'h out of college,',
 'of college, Bar',
 'ollege, Bar',
 'llege, Ba',
 'ege, Barr',
 'arry the ',
 'the Bee (',
 'e (Jerry ',
 ' (Jerry S',
 'rry Seinf',
 'einfeld) ',
 'feld) finds the',
 'inds the prospec',
 'e prospec',
 'pect of w',
 't of work',
 'of workin',
 ' working ',
 'working w',
 'ng with honey u',
 'g with hone',
 ' with honey',
 'h honey unins',
 'inspiring. H',
 'nspiring.',
 'ng. He flies o',
 's outside',
 'side the ',
 'ide the hive f',
 'e for the',
 'the first',
 'e first tim',
 'first time ',
 ' time and t',
 'e and tal',
 'and talks to a',
 'alks to a',
 'ks to a human',
 ' to a huma',
 ' human (R',
 'n (Renée ',
 'Renée Zellweger),',
 'enée Zell',
 'ée Zellweger',
 'ger), bre',
 'er), brea',
 ' breaking',
 'reaking a',
 'ing a car',
 'a cardinal rule',
 'rdinal rule of hi',
 'of his sp',
 'f his species. ',
 'ies. Barry',
 's. Barry ',
 '. Barry learn',
 'rry learn',
 'learns that h',
 'rns that ',
 's that hu',
 'that

In [4]:
ref = "The sly brown fox jumps swiftly over the lazy dog"
reads = chop_sequence(ref,6,11)
import random
random.shuffle(reads)
print(ref)
print(reads)

The sly brown fox jumps swiftly over the lazy dog
['r the ', 'over t', 'swiftly ove', 'ly over th', 'er the', 'own fox jum', 'brown ', 'The sly bro', 'n fox ', 'ox jum', 'tly ov', 'he lazy', 'lazy d', 'iftly ', 'x jump', 'jumps swift', 'y dog']


In [5]:
class Read:
  def __init__ (self,read,k_min=3):
    self.read = read
    self.k_min = k_min

  def __repr__(self):
    return self.read

  def __getitem__(self,key):
    return self.read[key]

  def __len__(self):
    return len(self.read)

  def partition(self,sep):
    return self.read.partition(sep)
  
  def startswith(self,key):
    return self.read.startswith(key)

  def endswith(self,key):
    return self.read.endswith(key)

  def find(self,key):
    return self.read.find(key)

  def __contains__(self,item):
    return item in self.read

  def is_continuous_with(self,other,root):
    min_pre = min(len(self.read.partition(root)[0]),len(other.read.partition(root)[0]))
    min_suf = min(len(self.read.partition(root)[2]),len(other.read.partition(root)[2]))
    return self.read[self.read.find(root)-min_pre:self.read.find(root)+len(root)+min_suf] == other.read[other.read.find(root)-min_pre:other.read.find(root)+len(root)+min_suf]

  def continues_to(self,other,root):
    return self.read.partition(root)[0].endswith(other.read.partition(root)[0])
  
  def continues_from(self,other,root):
    return self.read.partition(root)[2].startswith(other.read.partition(root)[2]) 

In [7]:
class Segment:
  def __init__(self,prefix,root,suffix,read):
    self.root = root
    self.read = read
    self.prefix = prefix
    self.suffix = suffix
    
  def __repr__(self):
    return '{}{}{}'.format(self.prefix.root + ' -> ' if self.prefix else '',self.root,' -> ' + self.suffix.root if self.suffix else '')

  def __hash__(self):
    return hash(self.root)

  def __eq__(self,other):
    return self.root == other.root

  def __len__(self):
    return len(self.root)

  def __getitem__(self,key):
    return self.root[key]

In [12]:
# v1
class Sequitur:
  def __init__(self,reads,k_min=3):
    self.k_min = k_min
    self.reads = list(map(Read,reads))
    self.transitions = {}
    self.ends = []
    self.seq = ''
    for read in self.reads:
      if read.read == ', Barry t':
        print()
      cont = False
      seg = None
      i = 1
      l = self.k_min
      if len(self.ends) > 0:
        # find connection
        for segment in self.ends:
          # read contained in longer read that was already added
          if read.read in self.transitions[segment].read:
            cont = True
            break
          for r in range(len(segment),self.k_min-1,-1):     
            # please leave this alone kanala
            if self.transitions[segment].read.continues_to(read,segment[-r:]) and read.continues_from(self.transitions[segment].read,segment[-r:]):
              # read connects to an open suffix
              seg = self.transitions[segment]
              if self.transitions[segment].prefix and self.transitions[segment].suffix: self.ends.remove(segment)
              l = read.find(segment) + len(segment)
              i = read.find(segment) + 1
              break
            # if segment in read and self.transitions[segment].read.continues_from(read,segment[-r:]) and read.continues_to(self.transitions[segment].read,segment[-r:]):
            #   print("segment:",segment[:r])
            #   print("read:",read)
            #   print("self.transitions[segment].read:",self.transitions[segment].read)
            #   print("------------------------------------------------------------")
          if seg: break
      if cont:
        continue
      if not seg:
        new_entry = True
        a = 0
        while read[:self.k_min+a] in self.transitions:
          # check for continuity
          if self.transitions[read[:self.k_min+a]].read.continues_from(read,read[:self.k_min]) or read.continues_from(self.transitions[read[:self.k_min+a]].read,read[:self.k_min]):
            # read contained in overlapping reads
            # that have already been added
            new_entry = False
            break
          a += 1
        if new_entry:
          seg = Segment(None,read[:self.k_min+a],None,read)
          self.transitions[read[:self.k_min+a]] = seg
          self.ends += [read[:self.k_min+a]]
        else:
          continue
      s = 0
      p = 0
      while l < len(read):
        if read[i+p:i+p+self.k_min+s] in self.transitions:
          # if self.transitions[read[i+p:i+p+self.k_min+s]].read.continues_from(read,read[i+p:i+p+self.k_min+s]) and read.continues_to(self.transitions[read[i+p:i+p+self.k_min+s]].read,read[i+p:i+p+self.k_min+s]):
          if self.transitions[read[i+p:i+p+self.k_min+s]].read.is_continuous_with(read,read[i+p:i+p+self.k_min+s]) == 'T':
          # if read.continues_to(self.transitions[read[i+p:i+p+self.k_min+s]].read,read[i+p:i+p+self.k_min+s]) and self.transitions[read[i+p:i+p+self.k_min+s]].read.continues_from(read,read[i+p:i+p+self.k_min+s]):
            self.transitions[read[i+p:i+p+self.k_min+s]].prefix = seg
            if read[i+p:i+p+self.k_min+s] in self.ends and self.transitions[read[i+p:i+p+self.k_min+s]].prefix and self.transitions[read[i+p:i+p+self.k_min+s]].suffix: self.ends.remove(read[i+p:i+p+self.k_min+s])
            seg.suffix = self.transitions[read[i+p:i+p+self.k_min+s]]
            if seg.prefix and seg.suffix: self.ends.remove(seg.root)
            curr = self.transitions[read[i+p:i+p+self.k_min+s]]
            while curr.suffix and curr.suffix.root in read:
              curr = curr.suffix
            if curr.suffix:
              break
            seg = curr
            seg.read = read
            l = read.find(curr.root) + len(curr.root)
            i = read.find(curr.root) + 1
            continue
          if i + abs(p) + s + self.k_min < len(read):
            s += 1
          else:
            p -= 1
            s += 1
            self.ends.remove(seg.root)
            self.transitions.pop(seg.root)
            seg.prefix.suffix = None
            seg = seg.prefix
            self.ends += [seg.root]
        else:
          self.transitions[read[i+p:i+p+self.k_min+s]] = Segment(seg,read[i+p:i+p+self.k_min+s],None,read)
          seg.suffix = self.transitions[read[i+p:i+p+self.k_min+s]]
          if seg.prefix and seg.suffix: self.ends.remove(seg.root)
          seg = self.transitions[read[i+p:i+p+self.k_min+s]]
          self.ends += [read[i+p:i+p+self.k_min+s]]
          l = read.find(read[i+p:i+p+self.k_min+s]) + len(read[i+p:i+p+self.k_min+s])
          i = read.find(read[i+p:i+p+self.k_min+s]) + 1
          s = 0
          p = 0

  def __repr__(self):
    return str(self.transitions)

  def construct(self):
    self.seq = 'Fre'
    curr = self.transitions[self.seq].suffix
    while curr:
        self.seq += curr.root.partition(self.seq[-(self.k_min-1):])[2]
        curr = curr.suffix
    return self.seq

In [13]:
seq = Sequitur(reads)

In [14]:
seq.transitions

{'Fre': Fre -> res,
 'res': Fre -> res -> esh,
 'esh': res -> esh -> sh ,
 'sh ': esh -> sh  -> h o,
 'h o': sh  -> h o ->  ou,
 ' ou': h o ->  ou -> out,
 'out':  ou -> out -> ut ,
 'ut ': out -> ut  -> t o,
 't o': ut  -> t o ->  of,
 ' of': t o ->  of -> of ,
 'of ':  of -> of  -> f c,
 'f c': of  -> f c ->  co,
 ' co': f c ->  co -> col,
 'col':  co -> col -> oll,
 'oll': col -> oll -> lle,
 'lle': oll -> lle -> leg,
 'leg': lle -> leg -> ege,
 'ege': leg -> ege -> ge,,
 'ge,': ege -> ge, -> e, ,
 'e, ': ge, -> e,  -> , B,
 ', B': e,  -> , B ->  Ba,
 ' Ba': , B ->  Ba -> Bar,
 'Bar':  Ba -> Bar -> arr,
 'arr': Bar -> arr -> rry,
 'rry': arr -> rry -> ry ,
 'ry ': rry -> ry  -> y t,
 'y t': ry  -> y t ->  th,
 ' th': y t ->  th -> the,
 'the':  th -> the -> he ,
 'he ': the -> he  -> e B,
 'e B': he  -> e B ->  Be,
 ' Be': e B ->  Be -> Bee,
 'Bee':  Be -> Bee -> ee ,
 'ee ': Bee -> ee  -> e (,
 'e (': ee  -> e ( ->  (J,
 ' (J': e ( ->  (J -> (Je,
 '(Je':  (J -> (Je -> Jer,
 'Jer': 

In [15]:
seq.construct()

'Fresh out of college, Barry the Bee (Jerry Seinfeld) finds the prospect of working with honey uninspiring. He flies outside the hive for the first time and talks to a human (Renée Zellweger), breaking a cardinal rule of his species. Barry learns that humans have been stealing and eating honey for centuries, and he realizes that his true calling is to obtain justice for his kind by suing humanity for theft.'